# Python Hackathon August 2024 Team 3 - Code Rangers

In [57]:
#import required libraries
import numpy as np
import pandas as pd
import warnings

#ignore python warnings
warnings.filterwarnings('ignore')

1. Join all sport-related tables to produce one dataframe that has all athletes values. The final table should only have the same 5 columns: ID, time,power,RR, vo2. Write this dataframe to an excel file.

In [59]:
#Read data from excel worksheets
df_Kayaking =pd.read_excel("V02Max_Combined.xlsx",sheet_name="Kayaking")
df_Athletes=pd.read_excel("V02Max_Combined.xlsx",sheet_name="Athletes")
df_Triathlon=pd.read_excel("V02Max_Combined.xlsx",sheet_name="Triathlon")
df_Fencing=pd.read_excel("V02Max_Combined.xlsx",sheet_name="Fencing")

In [60]:
#concatenate all sports data to form a single dataframe
df_data = pd.concat([df_Fencing,df_Kayaking,df_Triathlon],ignore_index=True)

In [63]:
#verify data
df_data.describe()

,ID,time,RR,VO2,power
count,52062.000000,52062.00000,50914.000000,52062.000000,52062.000000
mean,9.518766,547.45914,443.583808,1.539104,106.528754
std,5.043902,386.29623,212.585991,0.799706,88.603627
min,1.000000,-219.87200,252.000000,0.069354,0.000000
25%,5.000000,244.29500,352.000000,0.972290,0.000000
50%,10.000000,542.03800,412.000000,1.419143,95.000000
75%,14.000000,822.13900,484.000000,2.043474,170.000000
max,18.000000,1504.05600,30500.000000,4.576774,335.000000


In [65]:
# Export Dataframe to Excel by eliminating index
df_data.to_excel("V02Max_Sports.xlsx",sheet_name="All_Sports",index=False)

2. Divide the rows into 3 parts: Warm up, GET(graded exercise test) and cool down using the data def. Label each row accordingly

In [67]:
# cooldown = time>0 and power =0
# warmup = time<0 and power =0
# GET(graded exercise test) =time>0 and power>0

df_data["stage"] = ""
df_data.loc[df_data["power"] == 0, "stage"] = "cooldown"
df_data.loc[df_data["time"] < 0, "stage"] = "warmup"
df_data.loc[df_data["power"] > 0, "stage"] = "GET"

In [69]:
# Verify top five rows
df_data.head()

,ID,time,RR,VO2,power,stage
0,1,-130.840,672.0,0.302008,0,warmup
1,1,-130.168,672.0,0.302008,0,warmup
2,1,-129.536,632.0,0.301336,0,warmup
3,1,-128.824,712.0,0.301336,0,warmup
4,1,-127.948,876.0,0.301336,0,warmup


In [71]:
#verify the count of rows for each stage
df_data["stage"].value_counts()

stage
GET         38733
cooldown     9088
warmup       4241
Name: count, dtype: int64

3. What was the average time each tested lasted excluding warm up for all athletes?

In [73]:
#get max time of individual athelete excluding warmup stage. calculate mean of max time.
avg_test_time = df_data[df_data["stage"] != "warmup"].groupby("ID")["time"].max().mean()
print("The average time each tested lasted excluding warm up for all athletes is ",avg_test_time)

The average time each tested lasted excluding warm up for all athletes is  1096.9759999999999


4. Athletes from which sport had the highest average cardiorespiratory fitness in terms of HRV?

In [76]:
df_RR = df_data.copy()
#calculated differences of RR_intervals
df_RR["HRV"]= df_RR.groupby("ID")["RR"].diff()
# Calculated HRV as standard deviation of RR_interval diffrences and updated to the athelete dataframe. 
df_Athletes = pd.merge(df_Athletes,df_RR.groupby("ID")["HRV"].std(), on="ID")

In [78]:
#calculated mean HRV value for each sport and displyed the highest average sport
high_HRV_sport=df_Athletes.groupby("sport")["HRV"].mean().sort_values(ascending=False).head(1)
print("The sport that has the highest average cardiorespiratory fitness in terms of HRV is",high_HRV_sport)

The sport that has the highest average cardiorespiratory fitness in terms of HRV is sport
kayak    170.640909
Name: HRV, dtype: float64


5. What was the average time each test lasted (including warm up) for all athletes?

In [81]:
#total test time calculated as difference between the maximum and minimum of time
total_time_taken= df_data.groupby("ID")["time"].max() - df_data.groupby("ID")["time"].min()
avg_total_time = total_time_taken.mean()
print("The average time each tested lasted including warm up for all athletes is ",avg_total_time)

The average time each tested lasted including warm up for all athletes is  1277.8059999999998


8. What are the names of the kayaking athletes?

In [84]:
#filterd athelets with sport as kayak and printed their names
df_Athletes_kayak = df_Athletes[df_Athletes["sport"]=="kayak"]
print("The names of the kayaking athletes are :")
print(df_Athletes_kayak["First Name"].values + " " + df_Athletes_kayak["Last Name"].values)

The names of the kayaking athletes are :
['Isabella Griffin' 'Caleb Dixon' 'Mia Carpenter' 'Harper Mitchell'
 'Mason Wallace' 'Logan Anderson']


16. Calculate the BMI of every athlete and display it as a list.

In [87]:
#calculated BMI as weight(kg)(height(meter))^2
df_Athletes["BMI"] = df_Athletes["Weight"]/(df_Athletes["Height"]/100)**2
print("The list of athletes and their BMI is :")
df_Athletes.reset_index()[["First Name","Last Name","BMI"]].values.tolist()

The list of athletes and their BMI is :


[['Olivia', 'Martinez', 20.716793333590523],
 ['Ethan', 'Reynolds', 21.414418533760095],
 ['Jackson', 'Harper', 24.642964831007923],
 ['Liam', 'Sanchez', 23.61111111111111],
 ['Ava', 'Thompson', 22.083477858711987],
 ['Sophia', 'Rodriguez', 22.968383459962887],
 ['Grace', 'Simmons', 18.29201101928375],
 ['Aiden', 'Bennett', 18.80624426078972],
 ['Chloe', 'Price', 18.433626033057852],
 ['Lily', 'Jordan', 18.768553097768837],
 ['Isabella', 'Griffin', 25.600772853520105],
 ['Caleb', 'Dixon', 25.680074054167036],
 ['Mia', 'Carpenter', 25.05300247842575],
 ['Harper', 'Mitchell', 21.305700899422046],
 ['Mason', 'Wallace', 21.464305029745812],
 ['Logan', 'Anderson', 21.253471802922352],
 ['Lucas', 'Foster', 16.395463510848124],
 ['Owen', 'Myers', 15.902900176253803]]